<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Variational_Autoencoder_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

variational Autoencoder 2

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import (
    layers,
    models,
    callbacks,
    utils ,
    metrics,
    losses,
    optimizers,
)
from scipy.stats import norm
import pandas as pd

In [ ]:
from vae_utils_2 import get_vector_from_label ,add_vector_to_images, morph_faces

0.Parameters

In [ ]:
IMAGE_SIZE = 64
CHANNELS = 3
BATCH_SIZE = 128
NUM_FEATURES = 64
Z_DIM = 200
LEARNING_RATE = 0.0005
EPOCHS = 10
BETA = 2000
LOAD_MODEL = False

1. Prepare the data

In [ ]:
#Load the data
train_data = utils.image_dataset_from_directory(
    "/app/data/celeba-dataset/img_align_celeba/img_align_celeba",
    labels = None,
    color_mode = "rgb",
    image_size = (IMAGE_SIZE , IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 42,
    interpolation = "bilinear",
)


In [ ]:
#Preprocess the data

def preprocess(img):
  img = tf.cast(img, "float32")/ 255.0
  return img

train = train_data.map(lambda x : preprocess(x))

In [ ]:
train_sample = sample_batch(train)


In [ ]:
train_Sampel = sample_batch(train)

#2. Build the variational autoencoder

In [ ]:
class Sampling(layers.Layer):
  def call(self, inputs):
    z_mean , z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape = (batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
#Encoder
encoder_input = layers.Input(
    shape= (IMAGE_SIZE , IMAGE_SIZE , CHANNELS) , name = "encoder_input"
)
x = layers.Conv2D(NUM_FEATURES , kernel_size = 3, strides = 2, padding = "same")(
    encoder_input
)

x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3, strides = 2, padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3, strides = 2 , padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES , kernel_size = 3, strides = 2, padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3, strides = 2 , padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
shape_before_flattening = K.int_shape(x)[1:]

x = layers.Flatten()(x)
z_mean = layers.Dense(Z_DIM , name = "z_mean")(x)
z_log_var = layers.Dense(Z_DIM , name= "z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = models.Model(encoder_input, [z_mean, z_log_var, z] , name= "encoder")
encoder.summary()

In [ ]:
#Decoder
decoder_input = layers.Input(shape=(Z_DIM,) , name = "decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.BatchNormalization()(x)
x = layers.LeakyRelu()(x)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES, kernel_size = 3, strides = 2, padding = "same"
)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES, kernel_size = 3, strides = 2, padding = "same"
)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES, kernel_size = 3, strides = 2 , padding = "same"
)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES, kernel_size = 3, strides = 2, padding = "same"
)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES, kernel_size = 3, strides = 2 , padding = "same"
)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
decoder_output = layers.Conv2DTranspose(
    CHANNELS, kernel_size = 3, strides = 1, activation = "sigmoid", padding = "same"
)(x)
decoder = models.Model(decoder_input, decoder_output)
decoder.summary()

In [ ]:
class VAE(models.Model):
  def __init__(self, encoder, decoder, **kwargs):
    super(VAE, self).__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.total_loss_tracker = metrics.Mean(name= "total_loss")
    self.reconstruction_loss_tracker = metrics.Mean(
        name = "reconstruction_loss"
    )
    self.kl_loss_tracker = metrics.Mean(name= "kl_loss")

    @property
    def metrics(self):
      return[
          self.total_loss_tracker,
          self.reconstruction_loss_tracker,
          self.kl_loss_tracker,
      ]

    def call(self, inputs):
      """ Call the model on a particular input"""
      z_mean , z_log_var , z = encoder(inputs)
      reconstruction = decoder(z)
      return z_mean , z_log_var , reconstruction

    def train_step(self, data):
      """ Step run during training"""
      with tf.GradientTape() as tape:
        z_mean,  z_log_var , recontruction = self(data , training = True)
        reconstruction_loss = tf.reduce_mean(
            BETA * losses.mean_squared_error(data, reconstruction)
        )
        kl_loss = tf.reduce_mean(
            tf.reduce_sum(
                -0.5
                * (1 +z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                axis = 1,
            )
        )
        total_loss = reconstruction_loss + kl_loss

      grads = tape.gradient(total_loss , self.trainable_weights)
      self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

      self.total_loss_tracker.update_state(total_loss)
      self.reconstruction_loss_tracker.update_state(reconstruction_loss)
      self.kl_loss_tracker.update_state(kl_loss)

      return{
          "loss": self.total_loss_tracker.result(),
          "reconstruction_loss": self.reconstruction_loss_tracker.result(),
          "kl_loss" :self.kl_loss_tracker.result(),
      }


    def test_Step(self, data):
      """ Step run during validation."""
      if isinstance(data, tuple):
        data = data[0]

      z_mean, z_log_var, reconstruction = self(data)
      reconstruction_loss = tf.reduce_mean(
          BETA * losses.mean_squared_error(data, reconstruction)
      )
      kl_loss = tf.reduce_mean(
          tf.reduce_sum(
              -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
              axis = 1,
          )
      )
      total_loss = reconstruction_loss + kl_loss

      return{
          "loss" : total_loss,
          "reconstruction_loss" : reconstruction_loss,
          "kl_loss" : kl_loss,
      }





In [ ]:
#Create a variational autoencoder
vae = VAE(encoder, decoder)

#3. Train the variational autoencoder

In [ ]:
#Compile the variational autoencoder
optimizer = optimizers.Adam(learning_rate = LEARNING_RATE)
vae.compile(optimizer = optimizer)

In [ ]:
#Create a model save chekpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint",
    save_weights_only = False,
    save_freq = "epoch",
    monitor = "loss",
    mode = "min",
    save_best_only = True,
    verbose = 0,
)
tensorboard_callback = callbacks.TensorBoard(log_dir = "./logs")

class ImageGenerator(callbacks.Callback):
  def __init__(self, num_img , latent_dim):
    self.num_img = num_img
    self.latent_dim = latent_dim

  def on_epoch_end(self, epoch , logs = None):
    random_latent_vectors = tf.random.normal(
        shape= (self.num_img, self.latent_dim)
    )
    generated_images = self.model.decoder(random_latent_vectors)
    generated_images *= 255
    generated_images.numpy()
    for i in range(self.num_img):
      img = utils.array_to_img(generated_images[i])
      img.save("./output/generated_img_%03d_%d.png" % (epoch , i))


In [ ]:
#Load old weights if required
if LOAD_MODEL:
  vae.load_weight("./models/vae")
  tmp = vae.predict(train.take(1))

In [ ]:
vae.fit(
    train,
    epochs = EPOCHS,
    callbacks = [
        model_checkpoint_callback,
        tensornoard_callback,
        ImageGenerator(num_img = 10 , latent_dim=Z_DIM),
    ]
)